# Informe Técnico: Metodología de Predicción de Vida Útil Restante (RUL) para Bogies

<a id="indice"></a>

## Índice

1. [Introducción y Planteamiento del Problema](#introduccion)
2. [Metodología de Preparación de Datos](#preparacion-datos)

   * [Resumen de Características Iniciales](#resumen-inicial)
   * [Estrategia de Imputación de Valores Nulos](#imputacion-nulos)
   * [Proceso de Corrección de Outliers](#outliers)
   * [Limpieza de la Variable Objetivo](#limpieza-target)
3. [Ingeniería de Características y Definición del Objetivo](#ingenieria-features)

   * [Definición de la Variable Objetivo: RUL_steps](#rul-steps)
   * [Características de Ingeniería Diseñadas](#features-disenadas)
4. [Diseño del Pipeline de Entrenamiento y Validación](#pipeline)
5. [Experimentación y Comparativa de Modelos](#experimentacion)
6. [Análisis del Modelo Final y Resultados](#analisis-final)
7. [Conclusiones](#conclusiones)

---

<a id="introduccion"></a>

## Introducción y Planteamiento del Problema



El objetivo principal de este proyecto es desarrollar un modelo de mantenimiento predictivo capaz de anticipar fallos en los bogies de los trenes, componentes críticos para la seguridad y la operatividad de la flota. Este informe detalla la metodología técnica completa empleada, abarcando desde el procesamiento inicial de los datos de sensores hasta la evaluación del modelo final.

El desafío central surgió al intentar abordar el problema como una tarea de clasificación directa para predecir fallos (`target_fault`). Un análisis inicial de los datos reveló un desbalance extremo en la variable objetivo: los registros correspondientes a fallos (`target_fault = 1`) representaban únicamente el 1.5% del conjunto de datos, mientras que el 98.5% restante correspondía a operaciones normales.

Para superar este obstáculo, el problema se reformuló como una tarea de regresión: predecir la Vida Útil Restante (RUL). Este enfoque proporciona una medida cuantitativa y continua de la degradación del componente, permitiendo análisis de tendencia y priorización de riesgos.

[⬆ Volver al índice](#indice)

---

<a id="preparacion-datos"></a>

## Metodología de Preparación de Datos



La construcción de un modelo predictivo robusto depende directamente de la calidad de los datos de entrada. Por ello, se aplicó un procedimiento sistemático de limpieza, imputación y validación.

[⬆ Volver al índice](#indice)

<a id="resumen-inicial"></a>

### Resumen de Características Iniciales



| Métrica                 | Valor   |
| ----------------------- | ------- |
| Número de filas         | 200,000 |
| Número de columnas      | 23      |
| Número de bogies únicos | 40      |

[⬆ Volver al índice](#indice)

<a id="imputacion-nulos"></a>

### Estrategia de Imputación de Valores Nulos

X

* **timestamp**: imputación mediante `ffill` y `bfill` agrupando por `bogie_id`.
* **curve_class**: imputación con la moda (`straight`).
* **speed_kmh, load_tons, humidity_pct**: imputación con la mediana.
* **Variables de temperatura**: imputación jerárquica (mediana por `bogie_id` y, si persisten nulos, mediana global).


<a id="outliers"></a>

### Proceso de Corrección de Outliers



| Variable                | Rango aplicado | Justificación                  |
| ----------------------- | -------------- | ------------------------------ |
| speed_kmh               | 0 – 160        | Rango operativo realista       |
| load_tons               | 0 – 25         | Carga física plausible         |
| humidity_pct            | 0 – 100        | Límite físico                  |
| external_temp_c         | 0 – 50         | Temperatura ambiente           |
| bogie_temp_c            | 25 – 60        | Operación normal               |
| wheel_temp_left/right_c | 24 – 60        | Fricción y frenado             |
| vibration_x/y/z_rms     | 0 – 10         | Valores físicamente plausibles |
| days_since_last_maint   | 0 – 30         | Ciclo realista                 |


<a id="limpieza-target"></a>

### Limpieza de la Variable Objetivo


Se eliminaron 3,000 filas con valores nulos en `target_fault` o `fault_type`, garantizando la integridad del aprendizaje supervisado.

[⬆ Volver al índice](#indice)

---

<a id="ingenieria-features"></a>

## Ingeniería de Características y Definición del Objetivo



La ingeniería de características permitió capturar de forma explícita patrones de degradación y condiciones operativas anómalas.


<a id="rul-steps"></a>

### Definición de la Variable Objetivo: RUL_steps



* **Cálculo**: número de registros restantes hasta el siguiente fallo para cada combinación `train_id + bogie_id`.
* **Ventaja clave**: independencia de la frecuencia de muestreo.
* **Rango observado**: 0 a 272 pasos.

<a id="features-disenadas"></a>

### Características de Ingeniería Diseñadas



* **Agregados por bogie**: medias y desviaciones estándar.
* **Diferencias y ratios**: desequilibrios entre sensores.
* **Indicadores temporales**: deltas y medias móviles.
* **Normalización por operación**: ratios respecto a carga y velocidad.
* **Riesgo acumulado**: métricas de estrés y desgaste.

[⬆ Volver al índice](#indice)

---

<a id="pipeline"></a>

## Diseño del Pipeline de Entrenamiento y Validación


* **Split de datos**: 80% entrenamiento / 20% validación.
* **Criterio clave**: división por `train_id` para evitar data leakage.

### Balanceo Basado en Riesgo

1. Categorización en cuatro niveles de riesgo según `RUL_steps`.
2. Undersampling selectivo manteniendo el 100% de los casos críticos.

Resultado: conjunto de entrenamiento balanceado de 117,987 filas.

### Características Finales

| Tipo            | Variables                                                                                                                                                                                                               |
| --------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| Numéricas (16)  | vibration_x/y/z_rms, bogie_temp_c, wheel_temp_left/right_c, speed_kmh, load_tons, external_temp_c, humidity_pct, days_since_inspection, bogie_health_score, vib_xy_ratio, vib_xz_ratio, vib_load_ratio, vib_speed_ratio |
| Categóricas (3) | operation_mode, track_condition, curve_class                                                                                                                                                                            |

[⬆ Volver al índice](#indice)

---

<a id="experimentacion"></a>

## Experimentación y Comparativa de Modelos



Métrica prioritaria: **MAE en casos críticos (RUL < 50)**.

| Modelo              | MAE Total | MAE RUL<50 | Notas                  |
| ------------------- | --------- | ---------- | ---------------------- |
| Regresión Lineal    | 33.60     | 19.32      | Baseline               |
| XGBoost             | 35.00     | 14.00      | Alta interpretabilidad |
| Gradient Boosting   | 35.67     | 12.88      | Mejora clara           |
| Random Forest       | 35.71     | 12.23      | Competitivo            |
| Random Forest + PCA | 35.87     | 12.10      | Mejor en críticos      |

La aplicación de PCA redujo las 19 variables a 13 componentes, conservando el 96.8% de la varianza.

[⬆ Volver al índice](#indice)

---

<a id="analisis-final"></a>

## Análisis del Modelo Final y Resultados



El modelo final es un **Random Forest Regressor + PCA**, optimizado para minimizar el error en escenarios críticos.

### Métricas

* MAE total: **35.87 pasos**
* MAE RUL < 50: **12.10 pasos**
* RMSE RUL < 50: **14.19 pasos**

### Interpretación Operativa

La precisión alcanzada permite anticipar fallos con suficiente margen para planificar mantenimiento basado en condición.

### Importancia de Características (modelo XGBoost)

1. bogie_health_score
2. vib_load_ratio
3. external_temp_c
4. vib_xy_ratio
5. vib_speed_ratio


[⬆ Volver al índice](#indice)

---

<a id="conclusiones"></a>

## Conclusiones


La reformulación del problema como regresión de RUL permitió superar el severo desbalance de clases y generar una solución de alto valor operativo. El modelo final predice la vida útil restante con un error medio reducido en escenarios críticos, habilitando un mantenimiento proactivo, más seguro y eficiente para la operación ferroviaria.

[⬆ Volver al índice](#indice)